This is based off of https://github.com/superlinked/chunking-research/blob/main/preprocess.py

In [3]:
import datasets
from datasets import load_dataset

dataset = load_dataset("hotpotqa/hotpot_qa", "fullwiki")
dataset

/Users/aidand/dev/hello-retrieval-eval/env/lib/python3.10/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/Users/aidand/dev/hello-retrieval-eval/env/lib/python3.10/site-packages/datasets/load.py:1486: FutureWarning: The repository for hotpotqa/hotpot_qa contains custom code which must be executed to correctly load the dataset. You can inspect the repository content at https://hf.co/datasets/hotpotqa/hotpot_qa
You can avoid this message in future by passing the argument `trust_remote_code=True`.
Passing `trust_remote_code=True` will be mandatory to load this dataset from the next major release of `datasets`.
  warnings.warn(


DatasetDict({
    train: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 90447
    })
    validation: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
    test: Dataset({
        features: ['id', 'question', 'answer', 'type', 'level', 'supporting_facts', 'context'],
        num_rows: 7405
    })
})

In [4]:
df = dataset['train'].to_pandas()
df.head()

,id,question,answer,type,level,supporting_facts,context
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine,comparison,medium,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'title': ['Oberoi family', 'The Oberoi Group'...","{'title': ['Ritz-Carlton Jakarta', 'Oberoi fam..."
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'title': ['Allie Goertz', 'Allie Goertz', 'Al...","{'title': ['Lisa Simpson', 'Marge Simpson', 'B..."
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'title': ['Peggy Seeger', 'Peggy Seeger', 'Ew...","{'title': ['Moloch: or, This Gentile World', '..."
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'title': ['Cadmium chloride', 'Ethanol'], 'se...","{'title': ['Cadmium chloride', 'Water blue', '..."


In [5]:
sample = df.sample(1)
sample['context'].iloc[0]

{'title': array(['Bloodletting (Concrete Blonde album)',
        'Concrete Blonde y Los Illegals', 'Second Motion Records',
        'Concrete Blonde',
        "Estrojam's Decibelle Music and Culture Festival",
        'Manifesto Records', 'Dubstar', 'Paul Thompson (musician)',
        'Harry Rushakoff', 'Concrete Blonde (album)'], dtype=object),
 'sentences': array([array(['Bloodletting is the third studio album by American alternative rock band Concrete Blonde.',
               ' It was released on May 15, 1990.',
               ' It marks a shift for the band towards gothic rock.'],
              dtype=object)                                                                                ,
        array(['An album produced as a joint effort between Johnette Napolitano and James Mankey, previously founding members of alternative rock band Concrete Blonde, and L.A. pachucho punk band Los Illegals.',
               ' It contains a blend of hard rock and Latin music.'], dtype=object)    

In [6]:
def scentences_all_populated(context):
	for sentences in context['sentences']:
		if not all(sentences):
			return False
	return True

def scentences_populated(context):
	for sentences in context['sentences']:
		if any(sentences):
			return True
	return False

non_true_df = df[~df['context'].apply(scentences_all_populated)]
display(non_true_df.head())
display(non_true_df.shape)
display(non_true_df['context'].sample(1).iloc[0])

,id,question,answer,type,level,supporting_facts,context
474,5ae625335542995703ce8b28,"What football club has won 3 UFEA Super Cups, ...",Liverpool Football Club,bridge,medium,"{'title': ['Hughie Clifford', 'Liverpool F.C.'...","{'title': ['List of FC Barcelona players', 'Su..."
568,5addf5935542995b365fab9e,"Which plant, Buddleja or Burchardia, is an her...",Burchardia,comparison,easy,"{'title': ['Buddleja', 'Burchardia', 'Burchard...","{'title': ['Burchardia', 'Buddleja alata', 'Bu..."
713,5adc05f35542996e685252ac,"Brisingr is the third novel in the ""Inheritanc...",Christopher Paolini,bridge,easy,"{'title': ['Brisingr', 'Inheritance Cycle'], '...","{'title': ['The Wheel of Time', 'Lloyd Alexand..."
857,5a8ba7d75542996e8ac88991,How long was the Underdog show presented by an...,six-week television series,bridge,hard,"{'title': ['The Underdog Show', 'Julian Clary'...","{'title': ['List of Underdog episodes', 'Julia..."
1333,5ae4f18755429960a22e0219,When was the Australian radio and television p...,29 October 1964,bridge,medium,"{'title': ['1 vs. 100 (Australian game show)',...","{'title': ['Lai Van Sam', 'Tim Shaw (salespers..."


(400, 7)

{'title': array(['National Bird (film)', "America's Heart and Soul",
        'Nekkeveld estate', 'Bird Studies Canada',
        'Andean cock-of-the-rock', 'List of birds of India',
        'Saint Helena plover',
        'A Field Guide to Australian Birds (Slater)',
        'List of birds of Pakistan', 'Himalayan monal'], dtype=object),
 'sentences': array([array(['National Bird is a 2016 documentary film directed by Sonia Kennebeck with executive producers Wim Wenders and Errol Morris.',
               ' The film focuses on three whistleblowers who talk about unmanned aerial vehicles commonly called drones.',
               ' It was shown at the Tribeca Film Festival and the Berlin Film Festival, and it was reviewed in "Variety" and "The Guardian".'],
              dtype=object)                                                                                                                     ,
        array(["America's Heart and Soul is a 2004 film produced by Blacklight Films and rel

In [7]:
import numpy as np


df = df.loc[
	df['context'].apply(scentences_populated)
].reset_index(drop=True)
df.head()
# empty_contexts = df[df['context'].apply(lambda x: all(x['sentences']))]
# empty_contexts
# empty_contexts = []
# for i, row in df.iterrows():
# 	if not np.logical_and(row['context']['sentences']):
# 		empty_contexts.append(i)
# empty_contexts

# empty_contexts = df[df['context'].apply(lambda x: all(x['sentences']))]
# empty_contexts



,id,question,answer,type,level,supporting_facts,context
0,5a7a06935542990198eaf050,Which magazine was started first Arthur's Maga...,Arthur's Magazine,comparison,medium,"{'title': ['Arthur's Magazine', 'First for Wom...",{'title': ['Radio City (Indian radio station)'...
1,5a879ab05542996e4f30887e,The Oberoi family is part of a hotel company t...,Delhi,bridge,medium,"{'title': ['Oberoi family', 'The Oberoi Group'...","{'title': ['Ritz-Carlton Jakarta', 'Oberoi fam..."
2,5a8d7341554299441c6b9fe5,Musician and satirist Allie Goertz wrote a son...,President Richard Nixon,bridge,hard,"{'title': ['Allie Goertz', 'Allie Goertz', 'Al...","{'title': ['Lisa Simpson', 'Marge Simpson', 'B..."
3,5a82171f5542990a1d231f4a,What nationality was James Henry Miller's wife?,American,bridge,medium,"{'title': ['Peggy Seeger', 'Peggy Seeger', 'Ew...","{'title': ['Moloch: or, This Gentile World', '..."
4,5a84dd955542997b5ce3ff79,Cadmium Chloride is slightly soluble in this c...,alcohol,bridge,medium,"{'title': ['Cadmium chloride', 'Ethanol'], 'se...","{'title': ['Cadmium chloride', 'Water blue', '..."


In [8]:
# df['context'] = df.context.apply(
# 	lambda x: ["".join(sub_list) for sub_list in x['sentences']]
# )
# df.head()


In [9]:

df['context'].sample(1).iloc[0]

{'title': array(['2003–04 Ottawa Senators season',
        'List of Ottawa Senators general managers', 'Erik Karlsson',
        'List of Ottawa Senators head coaches',
        '2011–12 Ottawa Senators season', '1997–98 Ottawa Senators season',
        'History of the Ottawa Senators (1992–)',
        'Ottawa Senators (senior hockey)', 'Viking Award',
        '2004–05 Ottawa Senators season'], dtype=object),
 'sentences': array([array(['The 2003–04 Ottawa Senators season was the 12th season of the Ottawa Senators of the National Hockey League (NHL).',
               ' This season would see the Senators again finish with over 100 points, finishing with 102, but this was good for only third in the tightly-contested division, as the Boston Bruins would have 104 and the Toronto Maple Leafs 103.',
               " Ottawa would meet Toronto in the first-round of the playoffs for the fourth time, where the Maple Leafs would win the series 4–3 to end the Senators' playoff hopes.",
             

In [10]:
df['context'] = df.context.apply(
	# They just use the first context
	lambda x: ["".join(sub_list) for sub_list in x['sentences']][0]
)

In [ ]:
df['context_length'] = df['context'].apply(len)

KeyError: 'contex_length'

In [12]:
df['context_length'].mean()

541.626079361394